<h1>Exercise 3 </h1>

<p>Aditya Iqbal Bagaskara</p>

<h2>Importing libraries and define functions </h2>

In [15]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from random import random, randint, choice
from random import choice
from skimage.transform import resize
from scipy.stats import multivariate_normal

def unpickle(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding="latin1")
    return dict

def class_acc(pred, gt):
    correctAnswer = 0
    correctLabel = []
    for i in range(0, len(pred)):
        if(label_names[pred[i]] == label_names[gt[i]]):
            correctLabel = np.append(correctLabel, label_names[gt[i]])
            corectAnswer = correctAnswer + 1
    
    accuracy = len(correctLabel)/len(gt)
    return accuracy

def merge(dict1, dict2):
    return(dict2.update(dict1))


<h2> Load the Training Data and Test Data </h2>



In [16]:
from collections import defaultdict
datadict = unpickle('cifar-10-batches-py/data_batch_1')
datadict2 = unpickle('cifar-10-batches-py/data_batch_2')
datadict3 = unpickle('cifar-10-batches-py/data_batch_3')
datadict4 = unpickle('cifar-10-batches-py/data_batch_4')
datadict5 = unpickle('cifar-10-batches-py/data_batch_5')
datadict_test = unpickle('cifar-10-batches-py/test_batch')

dd = defaultdict(list)

for d in (datadict, datadict2, datadict3, datadict4, datadict5): # you can list as many input dicts as you want here
    for key, value in d.items():
        dd[key].append(value)
        
X_train = dd["data"]
Y_train = dd["labels"]


X_train = np.concatenate((dd["data"][0],dd["data"][1],dd["data"][2],dd["data"][3],dd["data"][4]), axis=0)
Y_train = np.concatenate((dd["labels"][0],dd["labels"][1],dd["labels"][2],dd["labels"][3],dd["labels"][4]), axis=0)


X_test = datadict_test["data"]
Y_test = datadict_test["labels"]


labeldict = unpickle('cifar-10-batches-py/batches.meta')
label_names = labeldict["label_names"]

X_train = X_train.reshape(len(X_train), 3, 32, 32).transpose(0,2,3,1).astype("int64")
X_test = X_test.reshape(len(X_test), 3, 32, 32).transpose(0,2,3,1).astype("int64")

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

<H1>CIFAR-10 – Bayesian classifier (good) </H1>

In [17]:
def cifar10_color(X):
    Xf = np.zeros((X.shape[0], 3))
    for i in range(X.shape[0]):
        mu_r = np.mean(X[i][:,:,0])
        mu_g = np.mean(X[i][:,:,1])
        mu_b = np.mean(X[i][:,:,2])
        Xf[i] = (mu_r, mu_g, mu_b)
    return Xf

In [18]:
X_train_1 = cifar10_color(X_train)

In [19]:
def cifar10_naivebayes_learn(Xp,Y):
    
    airplane = []
    automobile = []
    bird = []
    cat = []
    deer = []
    dog = []
    frog = []
    horse = []
    ship = []
    truck = []
    
    
    Xf = np.zeros(len(Y))

    
    for i in range(0,len(Xp)):
        if(Y_train[i] == 0):
            airplane.append((Xp[i]))
        if (Y[i] == 1):
            automobile.append(Xp[i])
        if (Y[i]== 2):
            bird.append(Xp[i])
        if (Y[i] == 3):
            cat.append(Xp[i])
        if (Y[i] == 4):
            deer.append(Xp[i])
        if (Y[i] == 5):
            dog.append( Xp[i])
        if (Y[i] == 6):
            frog.append(Xp[i])
        if (Y[i] == 7):
            horse.append(Xp[i])
        if (Y[i] == 8):
            ship.append(Xp[i])
        if (Y[i] == 9):
            truck.append(Xp[i])
            
    airplane = np.array(airplane)
    automobile = np.array(automobile)
    bird = np.array(bird)
    cat = np.array(cat)
    deer = np.array(deer)
    dog = np.array(dog)
    frog = np.array(frog)
    horse = np.array(horse)
    ship = np.array(ship)
    truck = np.array(truck)
    Xf = (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)

    img_mean = np.zeros((len(Xf), 3))
    for i in range(0, len(img_mean)):
        img_mean[i][0] = np.mean(Xf[i][:,0]) 
        img_mean[i][1] = np.mean(Xf[i][:,1]) 
        img_mean[i][2] = np.mean(Xf[i][:,2]) 
    
    img_std = np.zeros((len(Xf), 3))
    for i in range(0, len(img_std)):
        img_std[i][0] = np.std(Xf[i][:,0]) 
        img_std[i][1] = np.std(Xf[i][:,1]) 
        img_std[i][2] = np.std(Xf[i][:,2]) 
    
    total = 0
    for i in range(0, len(Xf)):
        total = total + Xf[i].shape[0] 
        
        
    pos = [len(x)/total for x in Xf]
    pos = np.array(pos)
        
    
    return img_mean, img_std, pos

In [20]:
mu, sig, pos = cifar10_naivebayes_learn(X_train_1, Y_train)

In [21]:
def normal_dist(x , mean , sd):
    prob_density = (1/ (sd*((2*np.pi)**(0.5)))) *  (np.exp( (-1/2)*(((x-mean)/sd)**2 )))
    return prob_density

def cifar10_classifier_naivebayes(x,mu,sigma,p):
    
    denominator = []
    for k in range(0,len(p)):
        dist = ((normal_dist(x[0], mu[k][0], sigma[k][0]))*((normal_dist(x[1], mu[k][1], sigma[k][1])))*((normal_dist(x[2], mu[k][2], sigma[k][2]))))*p[k]
        denominator.append(dist)
        
    denominator = np.sum(np.array(denominator))
    
    
    
    prob_array = []
    for i in range(0,len(p)):
        rnd = (normal_dist(x[0], mu[i][0], sigma[i][0]))
        gnd = (normal_dist(x[1], mu[i][1], sigma[i][1]))
        bnd = (normal_dist(x[2], mu[i][2], sigma[i][2]))
        posterior = p[i]
        
        
        d1 = (rnd * gnd * bnd * posterior)/denominator
        prob_array.append(d1)
        
    
    prob_array = np.array(prob_array) 
    
    
    return np.argmax(prob_array)

In [22]:
naive_bayes_result_train = []

for x in range(0, X_train_1.shape[0]):
    naive_bayes_result_train.append(cifar10_classifier_naivebayes(X_train_1[x], mu, sig, pos))
naive_bayes_result_train = np.array(naive_bayes_result_train)

In [23]:
naive_bayes_result_test = []

X_test_1 = cifar10_color(X_test)

for x in range(0, X_test_1.shape[0]):
    naive_bayes_result_test.append(cifar10_classifier_naivebayes(X_test_1[x], mu, sig, pos))
naive_bayes_result_test = np.array(naive_bayes_result_test)

In [24]:
accuracy_2 = class_acc(naive_bayes_result_test, Y_test)

print(f'Accuracy for task 1 test is {accuracy_2*100}%')

Accuracy for task 1 test is 19.470000000000002%


<h1>CIFAR-10 – Bayesian classifier (better) </h1>

In [25]:
def cifar10_bayes_learn_2(Xf,y):
    
    img_by_class = [[0] * 3 for i in range(0,10)]
    
    img_mean = [[0] * 3 for i in range(0,10)]
    
    img_mean = np.array(img_mean)
    
    
    for i in range(0,len(Xf)):
        img_class = y[i]
        for j in range(0,3):
            img_by_class[img_class][j] = np.append(img_by_class[img_class][j],Xf[i][j])
            
    
    img_cov =  np.zeros((10,3,3))
    
    for i in range(0,img_mean.shape[0]):
        for j in range(0,3):
            img_mean[i][j] = np.mean(img_by_class[i][j])
        img_cov[i] = np.cov((img_by_class[i]))
        
        
        
    total = 0
    for i in range(0, len(Xf)):
        total = total + Xf[i].shape[0] 
    pos = [len(x)/total for x in Xf]
    pos = np.array(pos)
        
    return img_mean, img_cov, pos

def cifar10_classifier_bayes_2(x,mu,sigma,p):
    prob_arr = np.zeros(10)
    
    for i in range(0,10):
        prob_arr[i] = (multivariate_normal.pdf(x,mu[i],sigma[i]))*p[i]
        
    pred_label = np.argmax(prob_arr)
    
    return pred_label  



In [26]:
X_train_2 = cifar10_color(X_train)
X_test_2 = cifar10_color(X_test)

mu2, sig2, pos2 = cifar10_bayes_learn_2(X_train_2, Y_train)

In [27]:
label_predict = np.zeros(len(X_test_2))
for i in range(0, len(X_test_2)):
    label_predict[i] = cifar10_classifier_bayes_2(X_test_2[i], mu2, sig2, pos2)

label_predict = label_predict.astype(int)

In [28]:
accuracy_3 = class_acc(label_predict, Y_test)

print(f'Accuracy for task 2 test is {accuracy_3*100}%')

Accuracy for task 2 test is 24.5%


<p> The accuracy for task 2 is slightly higher than task 1 since we are trying to find the correlation between those RGB colors. Providing covariane matrices as parameter can explains the relation between those values better in terms of their existence with other colors (for example to construct another color) rather than like the first classification when we just do each color independently </p>